In [1]:
from meteostat import Station, hourly
from datetime import datetime

station = Station("42867")  # Nagpur Sonegaon

start = datetime(2023, 1, 1)
end   = datetime.now()

data = hourly(station, start, end).fetch()

print(data.head())
print(data.shape)


                     temp  rhum  prcp  snwd  wdir  wspd  wpgt    pres  tsun  \
time                                                                          
2023-01-01 00:00:00  16.4    70   0.0  <NA>    20   3.6  <NA>  1017.0  <NA>   
2023-01-01 01:00:00  18.0    64   0.0  <NA>   360   3.6  <NA>  1019.0  <NA>   
2023-01-01 02:00:00  18.0    64   0.0  <NA>    30   3.6  <NA>  1020.0  <NA>   
2023-01-01 03:00:00  19.0    54   0.0  <NA>    50   5.4  <NA>  1019.8  <NA>   
2023-01-01 04:00:00  21.0    53   0.0  <NA>    30   3.6  <NA>  1021.0  <NA>   

                     cldc  coco  
time                             
2023-01-01 00:00:00     0     5  
2023-01-01 01:00:00     0     5  
2023-01-01 02:00:00     0     5  
2023-01-01 03:00:00     0     5  
2023-01-01 04:00:00     0     5  
(26615, 11)


In [2]:
df=data[['temp']].copy()

df=df.asfreq("H")

df=df.interpolate()

df.shape


C:\Users\kalas\AppData\Local\Temp\ipykernel_26916\3281332824.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df=df.asfreq("H")


(26615, 1)

In [3]:
from sklearn.preprocessing import MinMaxScaler
mx=MinMaxScaler()
X=df[['temp']].dropna().values
data_scaled=mx.fit_transform(X)
len(data_scaled)

26615

In [4]:
data_scaled[25-24:25].shape

(24, 1)

In [5]:
import numpy as np
def create_sequences(series,window):
    X,y=[],[]
    for i in range(window,len(series)):
        X.append(series[i-window:i])
        y.append(series[i,0])
    return np.array(X),np.array(y)

X,y=create_sequences(data_scaled,24)


In [6]:
split = int(0.8 * len(X))

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [7]:
import keras
from keras import Sequential
from keras.layers import LSTM,Dense,Input

model=Sequential([
    Input(shape=(24, 1)),
    LSTM(50),
    Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mse'
)


c:\Users\kalas\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [8]:
model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_batch_size=(y_train,y_test)
)

Epoch 1/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0062
Epoch 2/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0014
Epoch 3/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0013
Epoch 4/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0012
Epoch 5/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0011
Epoch 6/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0011
Epoch 7/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0011
Epoch 8/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0010
Epoch 9/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0010
Epoch 10/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0010
Epoch 11/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0010
Epoch 12/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 9.9306e-04
Epoch 13/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 9.8553e-04
Epoch 14/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 9.8975e-04
Epoch 15/20
665/665 ━━━━━━━━━━━━━━━━━━━━ 3s 4

In [ ]:
model.save(r"model/temp_lstm.h5")

import joblib
joblib.dump(mx,r"model/temp_scaler.pkl")


c:\Users\kalas\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


ValueError: Could not deserialize 'keras.metrics.mse' because it is not a KerasSaveable subclass

In [10]:
pred_scaled = model.predict(X_test)

pred_temp = mx.inverse_transform(pred_scaled)
actual_temp = mx.inverse_transform(y_test.reshape(-1, 1))


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [5]:
from keras.models import load_model

model = load_model(
    r"C:\Users\kalas\Desktop\desktopcopy\coding\ML_Projects\predict_weather\Predict_weather\model\temp_lstm.h5",
    compile=False
)

model.save(
    r"C:\Users\kalas\Desktop\desktopcopy\coding\ML_Projects\predict_weather\Predict_weather\model\temp_lstm.keras"
)